# 쿠팡 '노트북' 검색 후 데이터 스크래핑

In [317]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

## 참고자료

1. get 방식
- https://www.coupang.com/np/search?q=노트북&channel=user&component=&eventCategory= ..... 
- 위와 같이 /np/search? ~~~~ 값이 나옴
- ?뒤 부터 변수와 값을 의미
- 변수와 값은 &로 표시
- 쉽게 페이지를 요청할 수 있음
- 데이터 전송에 제약이 존재하여 큰 데이터는 보낼 수 없음


2. post 방식
- https body에 숨겨서 보내는 방식
- 보안데이터는 post방식으로 숨겨서 전송하는 방식이 안전(개인정보, 회원가입 등)
- 데이터 전송에 제약이 없음

# 1. BeautifulSoup로 스크래핑

In [328]:
url = 'https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page=1&rocketAll=false&searchIndexingToken=1=5&backgroundColor='
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
res = requests.get(url, headers = headers)
res.status_code
soup = BeautifulSoup(res.text, 'lxml')
print(res.text)


















<!doctype html>
<!--[if lte IE 7 ]><html class="lt-ie9 lt-ie8" lang="ko-KR"><![endif]-->
<!--[if IE 8 ]><html class="lt-ie9" lang="ko-KR"><![endif]-->
<!--[if (gte IE 9)|!(IE)]><!--><html lang="ko-KR"><!--<![endif]-->
<head>
    <meta charset="utf-8" />
    <meta name="google-site-verification" content="zaNrGtrOLMjglkziY2IvmL8dOXyCWHGArDHqFazJQVI" />
    <meta http-equiv="x-dns-prefetch-control" content="on" />
    <link rel="dns-prefetch" href="//cart.coupang.com" />
    <link rel="dns-prefetch" href="//assets.coupang.com" />
    <link rel="dns-prefetch" href="//assets2.coupang.com" />
    <link rel="dns-prefetch" href="//assets.coupangcdn.com" />
    <link rel="dns-prefetch" href="//asset1.coupangcdn.com" />
    <link rel="dns-prefetch" href="//private.coupang.com" />
    <link rel="dns-prefetch" href="//img1a.coupangcdn.com" />
    <link rel="dns-prefetch" href="//image1.coupangcdn.com" />
    <link rel="dns-prefetch" href="//thumbnail1.coupangcdn.com" />
    <link 

# 2. soup.find_all 함수로 조건에 맞는 값을 items라는 매개변수로 생성

- 여기서 re.compile('^search-product')을 사용한 이유는 광고가 붙지 않은 것은 search-product / 광고가 붙은 것은 search-product search-product__ad-badge이기 때문

In [329]:
items = soup.find_all('li', attrs={'class':re.compile("^search-product")})
items[0].find('div', attrs={'class':'name'}).get_text()

'HP 2021 노트북 15s, 스노우 화이트, 라이젠5 3세대, 256GB, 4GB, Free DOS, 15s-EQ1151AU'

# 3. items 변수 중 조건에 맞는 값 출력

### 1) 전체 출력

In [330]:
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
notebook = pd.DataFrame(columns = ['name','price','rate','rate_count'])

for i in items:
    name = i.find('div', attrs={'class':'name'}).get_text()                       # 제품명
    price = i.find('strong', attrs={'class':"price-value"}).get_text()            # 가격
    rate = i.find('em', attrs={'class':"rating"})                                 # 평점
    if rate:                                                                      # em 태그에 텍스트가 없는것(평점이 없음)이 존재
        rate = rate.get_text()                                                    # 따라서 if문으로 분리
    rate_count = i.find('span', attrs={'class':"rating-total-count"})             # 평점 수
    if rate_count:
        rate_count =rate_count.get_text()
    li_name.append(name)
    li_price.append(price)
    li_rate.append(rate)
    li_rate_count.append(rate_count)
            
li_notebook =[li_name, li_price, li_rate, li_rate_count]
for i in range(0,4):
    notebook.iloc[:,i]= li_notebook[i]
    
display(notebook)

,name,price,rate,rate_count
0,"HP 2021 노트북 15s, 스노우 화이트, 라이젠5 3세대, 256GB, 4GB...","490,780",5.0,(164)
1,"레노버 2021 ideapad 3D 14, 플래티넘 그레이, Slim3 14IML ...","369,000",5.0,(12)
2,"에이수스 2021 X415 14, 투명 실버, 코어i5 10세대, 256GB, 8G...","509,000",5.0,(114)
3,"디클 2021 노트북 14.1 + 키스킨, 화이트, 디클 클릭북 D141+ 플러스 ...","262,900",5.0,(1)
4,중고노트북 NT370E i5-4세대 win10 SSD 256GB RAM 8GB,"370,000",None,None
5,"10만원대 노트북 중고노트북, LG X NOTE R510/E500","159,000",4.0,(9)
6,"LG전자 2020 울트라 PC 15.6, 화이트, 라이젠3 3세대, 256GB, 8...","845,000",5.0,(1226)
7,"에이수스 2021 TUF DASH 15.6, 문라이트 화이트, 코어i7 11세대, ...","1,615,000",4.5,(69)
8,삼성 11세대 코어i5 윈도우10탑재 터치스크린 노트북 갤럭시북 Pro 360 NT...,"1,698,000",None,None
9,"에이수스 2020 VivoBook S15, 드리미 화이트, 코어i5 11세대, 51...","669,000",4.5,(23)


### 2) 광고 제외 출력

In [331]:
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
notebook = pd.DataFrame(columns = ['name','price','rate','rate_count'])

for i in items:
    ad_badge = i.find('span', attrs = {'class' : 'ad-badge-text'})
    if not ad_badge:
        name = i.find('div', attrs={'class':'name'}).get_text()                     
        price = i.find('strong', attrs={'class':"price-value"}).get_text()         
        rate = i.find('em', attrs={'class':"rating"})                              
        if rate:                                                                   
            rate = rate.get_text()                                                   
        rate_count = i.find('span', attrs={'class':"rating-total-count"})             
        if rate_count:
            rate_count =rate_count.get_text()
        li_name.append(name)
        li_price.append(price)
        li_rate.append(rate)
        li_rate_count.append(rate_count)

li_notebook =[li_name, li_price, li_rate, li_rate_count]
for i in range(0,4):
    notebook.iloc[:,i]= li_notebook[i] 

display(notebook)

,name,price,rate,rate_count
0,"HP 2021 노트북 15s, 스노우 화이트, 라이젠5 3세대, 256GB, 4GB...","490,780",5.0,(164)
1,"레노버 2021 ideapad 3D 14, 플래티넘 그레이, Slim3 14IML ...","369,000",5.0,(12)
2,"에이수스 2021 X415 14, 투명 실버, 코어i5 10세대, 256GB, 8G...","509,000",5.0,(114)
3,"디클 2021 노트북 14.1 + 키스킨, 화이트, 디클 클릭북 D141+ 플러스 ...","262,900",5.0,(1)
4,"LG전자 2020 울트라 PC 15.6, 화이트, 라이젠3 3세대, 256GB, 8...","845,000",5.0,(1226)
5,"에이수스 2021 TUF DASH 15.6, 문라이트 화이트, 코어i7 11세대, ...","1,615,000",4.5,(69)
6,"에이수스 2020 VivoBook S15, 드리미 화이트, 코어i5 11세대, 51...","669,000",4.5,(23)
7,"아이뮤즈 2018 스톰북 14.1, 로즈 골드, 셀러론, 64GB, 4GB, WIN...","279,000",4.5,(265)
8,"레노버 2021 IdeaPad Slim3 14, 플레티넘 그레이, A4, 128GB...","698,000",4.5,(139)
9,"레노버 2020 IdeaPad Slim1 14, 블루, 셀러론, 64GB, 4GB,...","305,970",4.5,(220)


### 3) 리뷰 100개 이상, 평점 4.5 이상 되는 것만 조회

In [333]:
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
notebook = pd.DataFrame(columns = ['name','price','rate','rate_count'])

for i in items:
    ad_badge = i.find('span', attrs = {'class' : 'ad-badge-text'})
    if not ad_badge:
        name = i.find('div', attrs={'class':'name'}).get_text()                       
        price = i.find('strong', attrs={'class':"price-value"}).get_text()

        rate = i.find('em', attrs={'class':"rating"})                                
        if rate:                                                                      
            rate = rate.get_text()
    
        rate_count = i.find('span', attrs={'class':"rating-total-count"})
        if rate_count:
            rate_count = rate_count.get_text()
            rate_count = rate_count[1:-1]            # 평점 출력 시 괄호를 제거해주기 위해
        
        # 리뷰 100개 이상, 평점 4.5 이상 출력
        if float(rate) >= 4.5 and int(rate_count) >=100 :
            li_name.append(name)
            li_price.append(price)
            li_rate.append(rate)
            li_rate_count.append(rate_count)
            
li_notebook =[li_name, li_price, li_rate, li_rate_count]
for i in range(0,4):
    notebook.iloc[:,i]= li_notebook[i]
    
display(notebook)

,name,price,rate,rate_count
0,"HP 2021 노트북 15s, 스노우 화이트, 라이젠5 3세대, 256GB, 4GB...","490,780",5.0,164
1,"에이수스 2021 X415 14, 투명 실버, 코어i5 10세대, 256GB, 8G...","509,000",5.0,114
2,"LG전자 2020 울트라 PC 15.6, 화이트, 라이젠3 3세대, 256GB, 8...","845,000",5.0,1226
3,"아이뮤즈 2018 스톰북 14.1, 로즈 골드, 셀러론, 64GB, 4GB, WIN...","279,000",4.5,265
4,"레노버 2021 IdeaPad Slim3 14, 플레티넘 그레이, A4, 128GB...","698,000",4.5,139
5,"레노버 2020 IdeaPad Slim1 14, 블루, 셀러론, 64GB, 4GB,...","305,970",4.5,220
6,"레노버 2021 ThinkPad E15, 블랙, 라이젠5 4세대, 256GB, 8G...","698,000",5.0,270
7,"레노버 2021 IdeaPad Slim3 15.6, 알틱 그레이, 코어i5 11세대...","789,000",4.5,466
8,"HP 2021 노트북 15s, 내츄럴 실버, 라이젠7 3세대, 512GB, 8GB,...","740,860",5.0,164
9,"Apple 2020 맥북 에어 13, 실버, M1, 256GB, 8GB, MAC O...","1,159,380",5.0,6269


### 4) 1~5p에서 광고가 아니며, 리뷰 100개 이상, 평점 4.5 이상 되는 것만 조회

In [335]:
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
li_page = []
li_name =[]
li_price = []
li_rate =[]
li_rate_count = []
li_link = []
notebook = pd.DataFrame(columns = ['page','name','price','rate','rate_count','link'])

for i in range(1,6):
    url = 'https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={}&rocketAll=false&searchIndexingToken=1=5&backgroundColor='.format(i)
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text, 'lxml')
    items = soup.find_all('li', attrs = {'class': re.compile('^search-product')})

    for item in items:
        ad_badge = item.find('span', attrs = {'class' : 'ad-badge-text'})
        if not ad_badge:
            page = i
            name = item.find('div', attrs = {'class' : 'name'}).get_text()
            price = item.find('strong', attrs = {'class' : 'price-value'}).get_text()
            
            rate = item.find('em', attrs = {'class' : 'rating'})
            if rate:
                rate = rate.get_text() 
                
            ## 위의 rate에서 else: continue를 사용했다면, 아래의 내용은 위 rate에 맞춰 들여쓰기를 할 필요가 없다!!
            
            
                rate_count = item.find('span', attrs = {'class' : 'rating-total-count'})
                if rate_count:
                    rate_count = rate_count.get_text()[1:-1]
                link = 'https://www.coupang.com/' + item.find('a', attrs = {'class' : 'search-product-link'})['href']
                if float(rate) >= 4.5 and int(rate_count) >= 100:
                    li_page.append(page)
                    li_name.append(name)
                    li_price.append(price)
                    li_rate.append(rate)
                    li_rate_count.append(rate_count)
                    li_link.append(link)

li_notebook =[li_page,li_name, li_price, li_rate, li_rate_count, li_link]
for i in range(0,6):
    notebook.iloc[:,i] = li_notebook[i]
        
display(notebook)

,page,name,price,rate,rate_count,link
0,1,"HP 2021 노트북 15s, 스노우 화이트, 라이젠5 3세대, 256GB, 4GB...","490,780",5.0,164,https://www.coupang.com//vp/products/523710176...
1,1,"에이수스 2021 X415 14, 투명 실버, 코어i5 10세대, 256GB, 8G...","509,000",5.0,114,https://www.coupang.com//vp/products/492329013...
2,1,"아이뮤즈 2018 스톰북 14.1, 로즈 골드, 셀러론, 64GB, 4GB, WIN...","279,000",4.5,265,https://www.coupang.com//vp/products/191066722...
3,1,"LG전자 2020 울트라 PC 15.6, 화이트, 라이젠3 3세대, 256GB, 8...","845,000",5.0,1226,https://www.coupang.com//vp/products/491910491...
4,1,"레노버 2020 IdeaPad Slim1 14, 블루, 셀러론, 64GB, 4GB,...","305,970",4.5,220,https://www.coupang.com//vp/products/223867010...
...,...,...,...,...,...,...
67,5,"삼성전자 2021 갤럭시북 프로 360 13.3 + S펜, 미스틱 네이비, 코어i5...","1,698,000",5.0,305,https://www.coupang.com//vp/products/542227223...
68,5,"레노버 2021 IdeaPad Slim3 14, Arctic Grey(그레이), 라...","988,000",4.5,139,https://www.coupang.com//vp/products/575025025...
69,5,"Apple 2020 맥북 프로 13, 스페이스 그레이, M1, 512GB, 16GB...","2,062,000",5.0,3015,https://www.coupang.com//vp/products/432248102...
70,5,"삼성전자 2021 노트북 플러스2 15.6, 블레이드 블랙, 코어i7 11세대, 5...","1,439,000",4.5,211,https://www.coupang.com//vp/products/609002591...


### 5) 엑셀파일로 저장

In [336]:
notebook.to_excel('C:/Users/Park/Desktop/data/sample.xlsx')